# 103 Spark - Movielens

The goal of this lab is to run some analysis on a different dataset, [MovieLens](https://grouplens.org/datasets/movielens/).

- Scala
    - [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
    - [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
    - [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)
- Python
    - [Spark programming guide](https://spark.apache.org/docs/3.5.0/rdd-programming-guide.html)
    - [All RDD APIs](https://spark.apache.org/docs/3.5.0/api/python/reference/api/pyspark.RDD.html)

**Download the dataset** from [here](https://big.csr.unibo.it/downloads/bigdata/ml-dataset.zip), unzip it and put it in the ```datasets/big``` folder.

- ml-movies.csv (<u>movieId</u>:Long, title:String, genres:String) 
    - genres are separated by pipelines  (e.g., "comedy|drama|action")
    - each movie is associated with many ratings

- ml-ratings.csv (<u>userId</u>:Long, <u>movieId</u>:Long, rating:Double, year:Int)
    - each rating is associated with many tags
    - ml-ratings-sample.csv is a small sample of ml-ratings.csv, useful for developing
- ml-tags.csv (<u>userId</u>:Long, <u>movieId</u>:Long, <u>tag</u>:String, year:Int) 

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Local Spark") \
    .config('spark.ui.port', '4040') \
    .getOrCreate()
sc = spark.sparkContext

sc

<SparkContext master=local appName=Local Spark>

In [3]:
path_to_datasets = "../../../../datasets/big/"

path_ml_movies = path_to_datasets + "ml-movies.csv"
path_ml_ratings = path_to_datasets + "ml-ratings-sample.csv"
path_ml_tags = path_to_datasets + "ml-tags.csv"

In [18]:
import re
from datetime import datetime
from typing import Optional, Tuple

class MovieLensParser:
    no_genres_listed = "(no genres listed)"
    comma_regex = re.compile(r',(?=(?:[^"]*"[^"]*")*[^"]*$)')
    pipe_regex = re.compile(r'\|(?=(?:[^"]*"[^"]*")*[^"]*$)')
    quotes = '"'

    @staticmethod
    def year_from_timestamp(timestamp: str) -> int:
        """Convert from timestamp (string) to year (int)."""
        ts = int(timestamp.strip())
        return datetime.utcfromtimestamp(ts).year

    @staticmethod
    def parse_movie_line(line: str) -> Optional[Tuple[int, str, str]]:
        """Parse a movie record into (movieId, title, genres)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            title = parts[1].strip()
            if title.startswith(MovieLensParser.quotes):
                title = title[1:]
            if title.endswith(MovieLensParser.quotes):
                title = title[:-1]
            return (int(parts[0].strip()), title, parts[2].strip())
        except Exception:
            return None

    @staticmethod
    def parse_rating_line(line: str) -> Optional[Tuple[int, int, float, int]]:
        """Parse a rating record into (userId, movieId, rating, year)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            year = MovieLensParser.year_from_timestamp(parts[3])
            return int(parts[0].strip()), int(parts[1].strip()), float(parts[2].strip()), year
        except Exception:
            return None

    @staticmethod
    def parse_tag_line(line: str) -> Optional[Tuple[int, int, str, int]]:
        """Parse a tag record into (userId, movieId, tag, year)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            year = MovieLensParser.year_from_timestamp(parts[3])
            return (int(parts[0].strip()), int(parts[1].strip()), parts[2].strip(), year)
        except Exception:
            return None


In [33]:
rddMovies = sc.textFile(path_ml_movies).map(MovieLensParser.parse_movie_line).filter(lambda x: x is not None)
rddRatings = sc.textFile(path_ml_ratings).map(MovieLensParser.parse_rating_line).filter(lambda x: x is not None)
rddTags = sc.textFile(path_ml_tags).map(MovieLensParser.parse_tag_line).filter(lambda x: x is not None)

## 103-1 Datasets exploration

Cache the datasets and answer the following questions:

- How many (distinct) users, movies, ratings, and tags?
- How many (distinct) genres?
- On average, how many ratings per user?
- On average, how many ratings per movie?
- On average, how many genres per movie?
- What is the range of ratings?
- Which years? (print an ordered list)
- On average, how many ratings per year?

In [25]:
rddMoviesCached = rddMovies.cache()
rddRatingsCached = rddRatings.cache()
rddTagsCached = rddTags.cache()

## 103-2 Compute the average rating for each movie

- Export the result to a file
- Do not start from cached RDDs
- Evaluate:
  - Join-and-Aggregate
  - Aggregate-and-Join
  - Aggregate-and-BroadcastJoin

In [30]:
path_output_avgRatPerMovie = "../../../../output/avgRatPerMovie"
# rdd.coalesce(1).toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

### Join-and-Aggregate

In [41]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],x[2])).\
    join(rddMoviesKV).\
    map(lambda x: ((x[0],x[1][1]),(x[1][0],1)) ).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    map(lambda x: (x[0][0], x[0][1], x[1][0]/x[1][1], x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

### Aggregate-and-Join

In [43]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],(x[2],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    mapValues(lambda x: (x[0]/x[1], x[1])).\
    join(rddMoviesKV).\
    map(lambda x: (x[0], x[1][1], x[1][0][0], x[1][0][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

### Aggregate-and-BroadcastJoin

In [44]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
bRddMovies = sc.broadcast(rddMoviesKV.collectAsMap())
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],(x[2],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    mapValues(lambda x: (x[0]/x[1], x[1])).\
    map(lambda x: (x[0],bRddMovies.value.get(x[0]),x[1][0],x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

## 103-3 Compute the average rating for each genre

Two possible workflows:

1. Pre-aggregation (3 shuffles)

  - Aggregate ratings by movieId
  - Join with movies and map to genres
  - Aggregate by genres
  
2. Join & aggregate (2 shuffles)

  - Join with movies and map to genres
  - Aggregate by genres

In [ ]:
path_output_avgRatPerMovie = "../../../../output/avgRatPerGenre"

for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()